In [1]:
import json
import os
import math
import numpy as np
import pandas as pd
import collections
import torch
#Specify the path to the output files
attackDir = 'attack/raw'
normalDir = 'normal_pred/raw'

In [2]:
#Open and store the list of filenames of the attack test cases
results = open('File_Names_Updated.txt','r')

In [3]:
fileNamesText = results.read()

In [4]:
fileNamesList = fileNamesText.split('\n')

In [5]:
fileNames = [fileName[2:-2] for fileName in fileNamesList]

In [6]:
#Extract scene title from the attack file name and store the corresponding normal prediction file name 
attackJsonDict = {}
normalJsonDict = collections.defaultdict(list)
normalFileName = {}
for fileName in fileNames:
    parts = fileName.split('-')
    frameID = parts[0]
    objectID = parts[1]
    attackName = parts[2].split('.')[0]
    attackJsonDict[fileName] = [frameID,objectID,attackName]
    normalFileName[fileName]="{}-{}.json".format(parts[0], parts[1])
    normalJsonDict[normalFileName[fileName]].append(fileName)
sceneNameList = list({str(value.split('.')[0]) for (key,value) in normalFileName.items()})

In [7]:
#Create a multi-indexed dataframe to store the error metrics against each prediction mode
sub_indices = ["normal", "ade", "fde", "front", "right", "rear", "left"]
columns = ["ADE", "FDE", "Front Deviation","Rear Deviation", "Right Deviation",  "Left Deviation"]
index = pd.MultiIndex.from_product([sceneNameList, sub_indices], names=["scene", "type"])
metrics_df = pd.DataFrame(index=index, columns=columns)
#metrics_df.index = metrics_df.index.set_levels(sceneNameList, level="row")

In [8]:
#Function to calculate all six errors
def calculateErrors(predict,future,observe):
    
    diff_square = np.square(predict-future)
    predict_trace = torch.from_numpy(predict)
    future_trace = torch.from_numpy(future)
    observe_trace = torch.from_numpy(observe)
    ADE = np.sum(np.sqrt(diff_square[:,0]+diff_square[:,1]))/len(predict_trace)
    FDE = np.sqrt(diff_square[-1:,0]+diff_square[-1:,1])[0]
    offset = predict_trace - future_trace
    direction = (future_trace - 
                 torch.cat(
                   (torch.reshape(observe_trace[-1,:], (1,2)), 
                    future_trace[:-1,:]), 0)).float()
    scale = torch.sqrt(torch.sum(torch.square(direction), 1)).float()
    right_direction = torch.matmul(
                        torch.tensor([[0., 1.], [-1., 0.]]).float(),
                        direction.t().float() / scale).t()
    average_right_distance = torch.sum(offset * right_direction) / predict_trace.shape[0]
    average_front_distance = torch.sum(offset * (direction.t().float() / scale).t()) / predict_trace.shape[0]
    right_distance = average_right_distance.item()
    left_distance = -average_right_distance.item()
    front_distance = average_front_distance.item()
    rear_distance = -average_front_distance.item()

    
    return [ADE, FDE, front_distance,rear_distance,right_distance,left_distance]

In [9]:
#Extract the observe trace, predict trace and future trace of the target vehicle as stored in the json files
#Frame 5 is the final prediction frame which has all the trace data post attack
#Calculate and store the metrics in the metrics_df dataframe
frame = 5
for sceneName in sceneNameList:
    
    normalFileName = "{}.json".format(sceneName)
    normalPath = os.path.join(normalDir,normalFileName)
    with open(normalPath, 'r') as f:
        normalContent = f.read()
    normalData = json.loads(normalContent)
    normalOutput = normalData['output_data']
    normalObjects = normalOutput[str(frame)]['objects']
    normal_predict_trace = np.array(normalObjects[str(sceneName.split('-')[1])]['predict_trace'])
    normal_observe_trace = np.array(normalObjects[str(sceneName.split('-')[1])]['observe_trace'])
    future_trace = np.array(normalObjects[str(sceneName.split('-')[1])]['future_trace'])
    normal_results = calculateErrors(normal_predict_trace,future_trace,normal_observe_trace)
    metrics_df.loc[(sceneName,'normal')] = normal_results
    
    for attack_mode in sub_indices[1:]:
        #attackFileName = fileName+'-'+attack_mode+'.json'
        attackFileName ="{}-{}.json".format(sceneName,attack_mode)
        attackPath = os.path.join(attackDir,attackFileName)

        with open(attackPath, 'r') as f:
            attackContent = f.read()
        attackData = json.loads(attackContent)
        attackOutput = attackData['output_data']
       
        attackObjects = attackOutput[str(frame)]['objects']
        attack_predict_trace = np.array(attackObjects[str(sceneName.split('-')[1])]['predict_trace'])
        #future_trace = attackObjects[str(attackJsonDict[attackFileName][1])]['future_trace']
        attack_observe_trace = np.array(attackObjects[str(sceneName.split('-')[1])]['observe_trace'])
        attack_results = calculateErrors(attack_predict_trace,future_trace,attack_observe_trace)
        metrics_df.loc[(sceneName,attack_mode)] = attack_results  

In [10]:
#Save the metrics dataframe
metrics_df.to_csv("All_Metrics.csv")

In [11]:
#Example data for case study
metrics_df.loc['0-24']

,ADE,FDE,Front Deviation,Rear Deviation,Right Deviation,Left Deviation
type,,,,,,
normal,2.950599,5.715799,2.280327,-2.280327,-1.372867,1.372867
ade,6.430954,11.743766,5.501614,-5.501614,-3.27492,3.27492
fde,6.426721,11.733865,5.494395,-5.494395,-3.278441,3.278441
front,6.533071,11.57958,5.616872,-5.616872,-3.322002,3.322002
right,2.413732,4.515473,1.969996,-1.969996,0.251792,-0.251792
rear,2.185283,2.567517,-1.055538,1.055538,-1.35162,1.35162
left,4.362663,7.825603,2.536129,-2.536129,-3.298795,3.298795


In [12]:
results_df = pd.read_csv('All_Metrics.csv')

In [13]:
#Group the rows by type of attack and calculate the average of each of the metrics for the prediction modes
format_out = results_df.groupby('type').mean().T.round(decimals=4)

C:\Users\mkjag\AppData\Local\Temp\ipykernel_17696\1762529664.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  format_out = results_df.groupby('type').mean().T.round(decimals=4)


In [14]:
#Store the results to a csv file
format_out.to_csv('Final_results.csv')